In [1]:
import requests
import datetime
import pandas as pd

In [5]:
# dt_start = datetime.datetime.strptime('2022-03-01 00:00:00', "%Y-%m-%d %H:%M:%S")
dt_start = datetime.datetime.strptime('2023-12-11 12:06:12', "%Y-%m-%d %H:%M:%S")
dt_end = datetime.datetime.strptime('2024-12-31 23:59:59', "%Y-%m-%d %H:%M:%S")
symbol = "AAPL"
limits_per_api_call = 1000

In [6]:
url = f"https://www.alphavantage.co/query"
params = {
	"function": "NEWS_SENTIMENT",
	"tickers": symbol,
	"apikey": "IUHOCUUMV8MWWMUM",
	"time_from": dt_start.strftime("%Y%m%dT%H%M"),
	"time_to": dt_end.strftime("%Y%m%dT%H%M"),
	"limit": limits_per_api_call,
	"sort": "EARLIEST",
}
response = requests.get(url, params=params)
data = response.json()

In [8]:
int(data['items'])

685

In [4]:
batch_news = []
for item in data["feed"]:
	news = {
		"symbol": symbol,
		"datetime": datetime.datetime.strptime(
			item["time_published"], "%Y%m%dT%H%M%S"
		),
		"title": item["title"],
		"source": item["source"],
		"summary": item["summary"],
		"length_summary": len(item["summary"]),
		"url": item["url"],
		"overall_sentiment": item["overall_sentiment_label"],
		"overall_sentiment_score": item["overall_sentiment_score"],
		"created_at": datetime.datetime.now(),  # Current timestamp for when the record is created
	}

	# Add ticker-specific sentiment if available
	if "ticker_sentiment" in item:
		for ticker_data in item["ticker_sentiment"]:
			if ticker_data["ticker"] == symbol:
				news["sentiment"] = ticker_data["ticker_sentiment_label"]
				news["sentiment_score"] = ticker_data[
					"ticker_sentiment_score"
				]
				news["relevance_score"] = ticker_data["relevance_score"]
				break

	batch_news.append(news)

KeyError: 'feed'

In [47]:
batch_news_df = pd.DataFrame(batch_news)
batch_news_df.shape

(678, 13)

In [48]:
batch_news_df.head(2)

,symbol,datetime,title,source,summary,length_summary,url,overall_sentiment,overall_sentiment_score,created_at,sentiment,sentiment_score,relevance_score
0,AAPL,2022-03-25 13:10:00,Is Most-Watched Stock Apple Inc. ( AAPL ) Wo...,Zacks Commentary,Zacks.com users have recently been watching Ap...,149,https://www.zacks.com/stock/news/1887424/is-mo...,Neutral,0.058284,2025-04-22 17:04:34.902832,Neutral,0.040413,0.49579
1,AAPL,2022-03-25 13:15:00,This Hyper-Successful Stock Is Totally Underes...,Motley Fool,Streaming business models primed for growth.,44,https://www.fool.com/investing/2022/03/25/this...,Somewhat-Bullish,0.342731,2025-04-22 17:04:34.902854,Somewhat-Bullish,0.342731,0.993371


In [49]:
batch_news_df.tail(2)

,symbol,datetime,title,source,summary,length_summary,url,overall_sentiment,overall_sentiment_score,created_at,sentiment,sentiment_score,relevance_score
676,AAPL,2022-04-15 14:16:00,"2 Reasons to Buy ExxonMobil, and 1 Reason to H...",Motley Fool,A sleeping giant is roaring back to life.,41,https://www.fool.com/investing/2022/04/15/2-re...,Somewhat-Bearish,-0.236782,2025-04-22 17:04:34.908092,Neutral,0.046902,0.266828
677,AAPL,2022-04-15 15:37:00,3 Things About Snap That Smart Investors Know,Motley Fool,Snap is one of the most popular social media a...,91,https://www.fool.com/investing/2022/04/15/snap...,Neutral,0.053056,2025-04-22 17:04:34.908098,Neutral,0.064945,0.189769
